<div align="center">

# **Tarea 2: Conteo Aproximado con Path Sampling Telescópico**

**Aproximación del Número de q-Coloraciones usando MCMC**

</div>

## Configuración y Librerías

In [ ]:
import numpy as np
import numba
from numba import njit, prange
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from tqdm.auto import tqdm
import time
import multiprocessing as mp
from multiprocessing import Pool, TimeoutError as MPTimeoutError
import itertools
import os
import warnings
warnings.filterwarnings('ignore')

# Configuración
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10
np.random.seed(42)

print(f"✓ Librerías cargadas")
print(f"  Numba version: {numba.__version__}")
print(f"  Threads disponibles: {numba.config.NUMBA_NUM_THREADS}")
print(f"  CPU count: {mp.cpu_count()}")

## Parámetros Globales del Experimento

In [ ]:
# ============================================================================
# CONFIGURACIÓN GLOBAL
# ============================================================================

# Parámetros del experimento
K_VALUES = list(range(3, 21))  # K = 3, 4, ..., 20
Q_VALUES = list(range(2, 16))   # q = 2, 3, ..., 15 (TODOS, incluso q < 2d+1)
EPSILON = 0.3                    # Precisión fija

# Límites duros
MAX_SIMULATIONS = 1000           # Máximo 1000 simulaciones
MAX_GIBBS_STEPS = 2000           # Máximo 2000 pasos por simulación
EXACT_TIMEOUT_SECONDS = 600      # 10 minutos para conteo exacto

# Configuración de ejecución
BATCH_SIZE = 100                 # Batch para Gibbs Sampler
RESULTS_DIR = 'results_telescopic/'

print(f"\n{'='*80}")
print(f"CONFIGURACIÓN DEL EXPERIMENTO")
print(f"{'='*80}")
print(f"K values:           {K_VALUES}")
print(f"q values:           {Q_VALUES}")
print(f"epsilon:            {EPSILON}")
print(f"Max simulaciones:   {MAX_SIMULATIONS}")
print(f"Max pasos Gibbs:    {MAX_GIBBS_STEPS}")
print(f"Timeout exacto:     {EXACT_TIMEOUT_SECONDS}s ({EXACT_TIMEOUT_SECONDS/60:.1f} min)")
print(f"Total experimentos: {len(K_VALUES)} K's × {len(Q_VALUES)} q's = {len(K_VALUES)*len(Q_VALUES)}")
print(f"{'='*80}\n")

## Módulo 1: Funciones Base Optimizadas (Gibbs Sampler)

Estas funciones implementan el Gibbs Sampler con 3 optimizaciones críticas:
1. **Zero allocations** en loops críticos
2. **Batch processing** con Numba prange
3. **Shared adjacency list** entre simulaciones

In [ ]:
# ============================================================================
# FUNCIONES BASE ULTRA-OPTIMIZADAS
# ============================================================================

@njit(cache=True)
def create_adjacency_list(K: int):
    """
    Crea lista de adyacencia para lattice K×K.
    
    Returns:
        adj_list: array (K²×4) con índices de vecinos (-1 si no existe)
        degrees: array (K²,) con número de vecinos
    """
    n = K * K
    adj_list = np.full((n, 4), -1, dtype=np.int32)
    degrees = np.zeros(n, dtype=np.int32)
    
    for i in range(K):
        for j in range(K):
            node_idx = i * K + j
            deg = 0
            
            if i > 0:
                adj_list[node_idx, deg] = (i - 1) * K + j
                deg += 1
            if i < K - 1:
                adj_list[node_idx, deg] = (i + 1) * K + j
                deg += 1
            if j > 0:
                adj_list[node_idx, deg] = i * K + (j - 1)
                deg += 1
            if j < K - 1:
                adj_list[node_idx, deg] = i * K + (j + 1)
                deg += 1
            
            degrees[node_idx] = deg
    
    return adj_list, degrees


@njit(fastmath=True, cache=True)
def initialize_valid_coloring(n: int, q: int, adj_list: np.ndarray, degrees: np.ndarray):
    """
    Inicializa coloración válida usando greedy algorithm.
    """
    coloring = np.zeros(n, dtype=np.int32)
    used = np.zeros(q, dtype=np.bool_)
    
    for node in range(n):
        used[:] = False
        for i in range(degrees[node]):
            neighbor = adj_list[node, i]
            if neighbor >= 0 and neighbor < node:
                used[coloring[neighbor]] = True
        
        for c in range(q):
            if not used[c]:
                coloring[node] = c
                break
    
    return coloring


@njit(fastmath=True, cache=True, inline='always')
def gibbs_sampler_step_optimized(coloring: np.ndarray, adj_list: np.ndarray,
                                  degrees: np.ndarray, q: int, n: int,
                                  node_order: np.ndarray, used: np.ndarray, 
                                  valid_colors: np.ndarray):
    """
    Un paso del Gibbs Sampler SIN ALLOCATIONS.
    """
    for idx in range(n):
        node = node_order[idx]
        
        used[:] = False
        for i in range(degrees[node]):
            neighbor = adj_list[node, i]
            if neighbor >= 0:
                used[coloring[neighbor]] = True
        
        n_valid = 0
        for c in range(q):
            if not used[c]:
                valid_colors[n_valid] = c
                n_valid += 1
        
        if n_valid > 0:
            chosen_idx = np.random.randint(0, n_valid)
            coloring[node] = valid_colors[chosen_idx]


@njit(fastmath=True, cache=True)
def run_single_gibbs_simulation(K: int, q: int, n_steps: int, seed: int,
                                 adj_list: np.ndarray, degrees: np.ndarray):
    """
    Ejecuta UNA simulación del Gibbs Sampler.
    """
    np.random.seed(seed)
    
    n = K * K
    coloring = initialize_valid_coloring(n, q, adj_list, degrees)
    
    node_order = np.arange(n, dtype=np.int32)
    used = np.zeros(q, dtype=np.bool_)
    valid_colors = np.empty(q, dtype=np.int32)
    
    for step in range(n_steps):
        np.random.shuffle(node_order)
        gibbs_sampler_step_optimized(coloring, adj_list, degrees, q, n, 
                                    node_order, used, valid_colors)
    
    return coloring


@njit(parallel=True, fastmath=True, cache=True)
def run_gibbs_batch_parallel(K: int, q: int, n_steps: int, seeds: np.ndarray,
                             adj_list: np.ndarray, degrees: np.ndarray):
    """
    Ejecuta BATCH de simulaciones en paralelo con Numba prange.
    """
    n_sims = len(seeds)
    n = K * K
    colorings = np.empty((n_sims, n), dtype=np.int32)
    
    for i in prange(n_sims):
        colorings[i] = run_single_gibbs_simulation(K, q, n_steps, seeds[i], 
                                                    adj_list, degrees)
    
    return colorings


print("✓ Gibbs Sampler optimizado implementado")
print("  - Zero allocations en loop crítico")
print("  - Batch processing con Numba prange")
print("  - Shared adjacency list")